In [21]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
import pandas as pd
import tensorflow as tf

In [5]:
def read_file(file):
    """Read a single file and return a dataframe"""
    return pd.read_csv(file, skipinitialspace=True)

In [22]:
df_options = read_file("data/benchmark_with_Pred_Naive_and_BS.csv")

In [23]:
df_low_money = df_options[df_options["Underlying_last"] / df_options["Strike"] < 0.97]
df_low_money.name = "<0.97"
df_mid_money = df_options[(df_options["Underlying_last"] / df_options["Strike"]).between(0.97, 1.03, inclusive=True)]
df_mid_money.name = "0.97-1.03"
df_high_money = df_options[df_options["Underlying_last"] / df_options["Strike"] > 1.03]
df_high_money.name = ">1.03"

In [26]:
df = pd.DataFrame(columns=['Moneyness', 'Maturity', 'B-S', 'Naive', 'LSTM'])
for dataframex in [df_low_money, df_mid_money, df_high_money]:
    i = 1
    for days in range(10):
        df_temp = dataframex[(df_options["Ttl"]).between(i, i + 89, inclusive=True)]
        bs = sqrt(tf.keras.metrics.mean_squared_error(df_temp["Price"], df_temp["BS"]))
        naive = sqrt(tf.keras.metrics.mean_squared_error(df_temp["Price"], df_temp["Naive"]))
        lstm = sqrt(tf.keras.metrics.mean_squared_error(df_temp["Price"], df_temp["Prediction"]))
        df.loc[len(df.index)] = [dataframex.name, str(i) + "-" + str(i + 89), bs, naive, lstm]
        i = i + 90

<ipython-input-26-3e712a2da552>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp = dataframex[(df_options["Ttl"]).between(i, i + 89, inclusive=True)]
<ipython-input-26-3e712a2da552>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp = dataframex[(df_options["Ttl"]).between(i, i + 89, inclusive=True)]
<ipython-input-26-3e712a2da552>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp = dataframex[(df_options["Ttl"]).between(i, i + 89, inclusive=True)]
<ipython-input-26-3e712a2da552>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp = dataframex[(df_options["Ttl"]).between(i, i + 89, inclusive=True)]
<ipython-input-26-3e712a2da552>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp = dataframex[(df_options["Ttl"]).between(i, i + 89, inclusive=True)]
<ipython-input-26-3e712a2da552>:5: UserWarning: Bo

In [27]:
print(df)

    Moneyness Maturity         B-S      Naive       LSTM
0       <0.97     1-90    4.204846   2.823519  17.463075
1       <0.97   91-180   13.669220   6.051504  11.031143
2       <0.97  181-270   20.239733   5.928463  11.565058
3       <0.97  271-360   29.471541  10.963838  17.533759
4       <0.97  361-450   38.887352  13.215093  20.913672
5       <0.97  451-540   47.104881  14.513897  17.966305
6       <0.97  541-630   53.419363  18.022719  33.112971
7       <0.97  631-720   56.501549  17.037032  24.690590
8       <0.97  721-810  101.021216  16.439628  60.525703
9       <0.97  811-900   86.392530  12.816688  15.634637
10  0.97-1.03     1-90   16.223921  16.951330  49.107860
11  0.97-1.03   91-180   44.846009  15.509885  18.408163
12  0.97-1.03  181-270   69.023797  15.983753  20.180332
13  0.97-1.03  271-360   86.402735  17.682173  26.617325
14  0.97-1.03  361-450  101.910640  17.036606  29.832996
15  0.97-1.03  451-540  113.318677  17.786849  26.519897
16  0.97-1.03  541-630  121.726

In [16]:
# Write to file
df_options.to_csv('data/benchmark_with_Naive_and_BS.csv', encoding='utf-8', index=False)

In [30]:
with open('results.tex', 'w') as tf:
     tf.write(df.to_latex())